In [13]:
import re
import sys
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style

In [14]:
with open('./res/lab1.json') as lab1_file:
    data = json.load(lab1_file)
chain = data['chain']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']

In [15]:
def getToken(token_index):
    global chain, service_words, operations, separators, constants, identifiers
    codes = ['W', 'O', 'S', 'C', 'I']
    tables = [service_words, operations, separators, constants, identifiers]
    return tables[codes.index(chain[token_index][0])][chain[token_index][1]]

In [12]:
getToken(0)

'void'